In [2]:
from gtts import gTTS
from pydub import AudioSegment
import random, os

## 캐시와 결과 저장을 위한 디렉터리 만들기

In [3]:
os.makedirs('samples', exist_ok=True)
os.makedirs('result', exist_ok=True)

## 입력 정의

In [14]:
string = '안녕하세요'
lang = 'ko'
random_factor = 0.35
normal_frame_rate = 44100

## 오디오 처리

In [17]:
result_sound = None

for i, letter in enumerate(string):
    # 사운드가 없을 때 
    if letter == ' ':
        new_sound = letter_sound._spawn(b'\x00' * (normal_frame_rate // 3), overrides={'frame_rate': normal_frame_rate})
    # 사운드가 있을 때
    else:
        # 캐시에 저장된 사운드가 아니면 TTS를 실행하여 소리를 저장한다.
        if not os.path.isfile('samples/%s.mp3' % letter):
            tts = gTTS(letter, lang=lang)
            tts.save('samples/%s.mp3' % letter)
        
        # 사운드 불러오기
        letter_sound = AudioSegment.from_mp3('samples/%s.mp3' % letter)
        
        # 샘플의 앞뒤 약간의 공백을 자른다.
        raw = letter_sound.raw_data[5000:-5000]
        
        # 피치를 랜덤한 값을 통해 글자마다 약간씩 다르게 조정되게한다.
        octaves = 2.0 + random.random() * random_factor
        frame_rate = int(letter_sound.frame_rate * (2.0 ** octaves))
        print('%s - octaves: %.2f, fr: %.d' % (letter, octaves, frame_rate))
        
        new_sound = letter_sound._spawn(raw, overrides={'frame_rate': frame_rate})
        
    # new sound 프레임 레이트를 normal_frame_rate로 변환
    new_sound = new_sound.set_frame_rate(normal_frame_rate)
    result_sound = new_sound if result_sound is None else result_sound + new_sound
    
result_sound

안 - octaves: 2.34, fr: 121615
녕 - octaves: 2.25, fr: 114537
하 - octaves: 2.29, fr: 117431
세 - octaves: 2.02, fr: 97364
요 - octaves: 2.19, fr: 109164


In [ ]:
result_sound.export('result/%s.mp3' % string, format='mp3')